<a href="https://colab.research.google.com/github/IllgamhoDuck/googleXproject/blob/master/cirq_with_tensornetwork_quantum_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Simulator with TensorNetwork
**Build a quantum simulator using Tensornetwork with Jax**



## Setting environment
- package install
- package import

In [0]:
!pip install --quiet protobuf==3.8.0 networkx==2.3 ply==3.4 cirq tensornetwork jax jaxlib nxpd pylatexenc

In [0]:
%matplotlib inline

# cirq - test and visualize
import cirq
from cirq import Circuit, Simulator
from cirq.devices import GridQubit, LineQubit
from cirq.contrib.qasm_import import circuit_from_qasm

# tensornetwork
import numpy as np
import jax
import tensornetwork as tn

# math
import math

# test
import os
import time
import random
from tqdm import tqdm_notebook as tqdm

In [0]:
class colors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    CEND      = '\33[0m'
    CBOLD     = '\33[1m'
    CITALIC   = '\33[3m'
    CURL      = '\33[4m'
    CBLINK    = '\33[5m'
    CBLINK2   = '\33[6m'
    CSELECTED = '\33[7m'

    CBLACK  = '\33[30m'
    CRED    = '\33[31m'
    CGREEN  = '\33[32m'
    CYELLOW = '\33[33m'
    CBLUE   = '\33[34m'
    CVIOLET = '\33[35m'
    CBEIGE  = '\33[36m'
    CWHITE  = '\33[37m'

    CBLACKBG  = '\33[40m'
    CREDBG    = '\33[41m'
    CGREENBG  = '\33[42m'
    CYELLOWBG = '\33[43m'
    CBLUEBG   = '\33[44m'
    CVIOLETBG = '\33[45m'
    CBEIGEBG  = '\33[46m'
    CWHITEBG  = '\33[47m'

    CGREY    = '\33[90m'
    CRED2    = '\33[91m'
    CGREEN2  = '\33[92m'
    CYELLOW2 = '\33[93m'
    CBLUE2   = '\33[94m'
    CVIOLET2 = '\33[95m'
    CBEIGE2  = '\33[96m'
    CWHITE2  = '\33[97m'

    CGREYBG    = '\33[100m'
    CREDBG2    = '\33[101m'
    CGREENBG2  = '\33[102m'
    CYELLOWBG2 = '\33[103m'
    CBLUEBG2   = '\33[104m'
    CVIOLETBG2 = '\33[105m'
    CBEIGEBG2  = '\33[106m'
    CWHITEBG2  = '\33[107m'

c = colors

## Introduction to Quantum simulator

1. What is quantum simulator?
2. How to build a circuit
3. How to check the result
4. Other useful features


### 1. **What is quantum simulator?**



It is a real quantum computer which can simulate the real quantum system.
At this colab it means different. Quantum simulator indicates a program simulates the quantum system in the classical computer without quantum computer. 

To express the quantum world we use qubit and unitary operator which is also called as quantum gate. The diagram that express the qubit and gate is quantum circuit like the picture below.

**Quantum circuit**

<img src="https://www.codeproject.com/KB/recipes/5160469/CircuitExample.png" width=600px>

*Vaughan, Daniel. “Quantum Computation Primer - Part 2.” CodeProject, CodeProject, 26 June 2019, www.codeproject.com/Articles/5160469/Quantum-Computation-Primer-Part-2.*

**Why do we use quantum simulator?**

Quantum simulator is very useful because we can simulate quantum circuit without using the slow and expensive quantum computer. If quantum circuit includes more than 16 qubits it is hard to find a place to simulate.

Also the simulator is not affect by the noise from real world. Quantum computer has a error correction method to protect the result being wrong by the noise but it is not perfect. But simulator will give us the correct result what it should have be given. So the simulator could be used to be a index to check the performance of quantum computer error correction method.

**Quantum simulator limitation**

There is a problem with using classic quantum simulator. When there is 1 qubit the statevector number is 2. When there is 2 qubit it goes up to 4. 3 qubit is 8. 

n qubit = $2^n$ statevector

    [How big will the memory needed?]
    30 qubits -> 1 GB
    40 qubits -> 1 TB
    50 qubits -> 1 PB
    ...

As the qubit size increase it is hard to simulate even for the supercomputer. So This is the limitation of the classic quantum simulator. It means we actually need a quantum computer.

At the below picture you can check the classicially simulatable qubit size.

<img src="https://regmedia.co.uk/2018/03/06/quantum_performance_curve.jpg" width=600px>

Chirgwin, Richard. “'Quantum Supremacy Will Soon Be Ours!', Says Google as It Reveals 72-Qubit Quantum Chip.” • The Register, The Register, 6 Mar. 2018, www.theregister.co.uk/2018/03/06/google_bristlecone_72_qubit_quantum_processsor/.

### 2. **How to build a quantum circuit?**

In this section we will learn how to build a quantum simulator step by step



In [0]:
#@title [step 0] - Quantum simulator code Just run :)
class QuantumSimulator():
    """Quantum simulator with tensornetwork"""
    def __init__(self, qbit_n):
        """
        Args:
            qbit_n: The number of total qubit size
            circuit: Where to store gate
        """
        assert qbit_n > 0, "Qubit size should be at least 1"
        self.qbit_n = qbit_n
        self.circuit = []

    def initialize_circuit(self):
        """
        Initialize the circuit to execute

        Args:
            qbits: Store the first initialized qubits
            measures: Store the edge that used for measure for each qubit
        """
        # Initialize qubit
        self.qbits = [tn.Node(np.array([1 + 0j, 0 + 0j])) for _ in range(self.qbit_n)]

        # Storing the Edge node that will be used to measure for each qubit
        self.measures = [self.qbits[i][0] for i in range(self.qbit_n)]
        self.amplitude = None
        self.result = {}
        self.result_prob = {}
    
    def connect_qubits(self):
        """
        Make the seperate tensornetworks to one tensornetwork using
        2 qubit identity gate
        CI - Control Identity
             [[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]]
        """
        for i in range(self.qbit_n - 1):
            self.add_ci(i, i + 1)

    def x(self, qbit_i):
        """
        Add X gate to specific 1 qubit index 
        x(qbit_i=0)
        """
        self.circuit.append(('x', qbit_i))

    def y(self, qbit_i):
        """
        Add Y gate to specific 1 qubit index 
        y(qbit_i=5)
        """
        self.circuit.append(('y', qbit_i))

    def z(self, qbit_i):
        """
        Add Z gate to specific 1 qubit index 
        z(qbit_i=3)
        """
        self.circuit.append(('z', qbit_i))

    def h(self, qbit_i):
        """
        Add H gate to specific 1 qubit index 
        h(qbit_i=2)
        """
        self.circuit.append(('h', qbit_i))

    def t(self, qbit_i):
        """
        Add T gate to specific 1 qubit index 
        t(qbit_i=0)
        """
        self.circuit.append(('t', qbit_i))

    def cx(self, qbit_c, qbit_t):
        """
        Add CX gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        cx(qbit_c=0, qbit_t=1)
        """
        self.circuit.append(('cx', qbit_c, qbit_t))

    def cy(self, qbit_c, qbit_t):
        """
        Add CY gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        cy(qbit_c=3, qbit_t=2)
        """
        self.circuit.append(('cy', qbit_c, qbit_t))

    def cz(self, qbit_c, qbit_t):
        """
        Add CZ gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        cz(qbit_c=2, qbit_t=4)
        """
        self.circuit.append(('cz', qbit_c, qbit_t))

    def ch(self, qbit_c, qbit_t):
        """
        Add CH gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        ch(qbit_c=1, qbit_t=3)
        """
        self.circuit.append(('ch', qbit_c, qbit_t))

    def add_x(self, qbit_i):
        X = tn.Node(np.array([[0, 1],
                              [1, 0]]))
        self.measures[qbit_i] ^ X[0]
        self.measures[qbit_i] = X[1]

    def add_y(self, qbit_i):
        Y = tn.Node(np.array([[0, 1j],
                              [-1j, 0]]))
        self.measures[qbit_i] ^ Y[0]
        self.measures[qbit_i] = Y[1]

    def add_z(self, qbit_i):
        Z = tn.Node(np.array([[1, 0],
                              [0, -1]]))
        self.measures[qbit_i] ^ Z[0]
        self.measures[qbit_i] = Z[1]

    def add_h(self, qbit_i):
        h_f = 1/math.sqrt(2)
        H = tn.Node(np.array([[h_f, h_f],
                              [h_f, -h_f]]))
        self.measures[qbit_i] ^ H[0]
        self.measures[qbit_i] = H[1]

    def add_t(self, qbit_i):
        e_j_pi = math.e ** ((1j * math.pi) / 4)
        T = tn.Node(np.array([[1, 0],
                              [0, e_j_pi]]))
        self.measures[qbit_i] ^ T[0]
        self.measures[qbit_i] = T[1]

    def add_cx(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        cx = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 0, 1],
                       [0, 0, 1, 0]])
        CX = tn.Node(cx.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CX[0]
        self.measures[qbit_t] ^ CX[1]
        self.measures[qbit_c] = CX[2]
        self.measures[qbit_t] = CX[3]

    def add_cy(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        cy = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 0, 1j],
                       [0, 0, -1j, 0]])
        CY = tn.Node(cy.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CY[0]
        self.measures[qbit_t] ^ CY[1]
        self.measures[qbit_c] = CY[2]
        self.measures[qbit_t] = CY[3]


    def add_cz(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        cz = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, -1]])
        CZ = tn.Node(cz.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CZ[0]
        self.measures[qbit_t] ^ CZ[1]
        self.measures[qbit_c] = CZ[2]
        self.measures[qbit_t] = CZ[3]

    def add_ch(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
            h_f: hadamard factor
        """
        h_f = 1/math.sqrt(2)
        ch = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, h_f, h_f],
                       [0, 0, h_f, -h_f]])
        CH = tn.Node(ch.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CH[0]
        self.measures[qbit_t] ^ CH[1]
        self.measures[qbit_c] = CH[2]
        self.measures[qbit_t] = CH[3]

    def add_ci(self, qbit_c, qbit_t):
        """
        Control Identity matrix for connect
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        ci = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]])
        CI = tn.Node(ci.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CI[0]
        self.measures[qbit_t] ^ CI[1]
        self.measures[qbit_c] = CI[2]
        self.measures[qbit_t] = CI[3]

    def generate_result(self):
        for i in range(2 ** self.qbit_n):
            # Generate statevector
            statevector = ""
            for shift in range(self.qbit_n):
                statevector = str((i >> shift) & 1) + statevector
            
            # Check the amplitude
            amp = self.amplitude[self.bitstring_to_int(statevector)]
            amp = complex(str(amp))

            if amp == 0:
                continue

            # TODO: Figure out how to get value out of jax array
            self.result[statevector] = amp 
            self.result_prob[statevector] = (amp*amp.conjugate()).real

    def bitstring_to_int(self, bitstring):
        int_list = []
        for i in bitstring[::-1]:
            int_list.append(int(i))
        return tuple(int_list)

    def execute_circuit(self):
        """
        gate is structed as following
        1 qubit gate - (gate_name, qubit_index)
        2 qubit gate - (gate_name, qubit_control_index, qubit_target_index)
        """

        # Connect the qubits
        for gate in self.circuit:
            if gate[0] in ['cx', 'cy', 'cz', 'ch']:
                exec('self.add_{}({}, {})'.format(gate[0], gate[1], gate[2]))
            else:
                exec('self.add_{}({})'.format(gate[0], gate[1]))

        nodes = tn.reachable(self.qbits[0])
        self.amplitude = tn.contractors.greedy(nodes, output_edge_order=self.measures).tensor

    def clear_circuit(self):
        self.circuit = []

    def change_qubit_size(self, size):
        assert size > 0, "Qubit size should be at least 1"
        self.qbit_n = size

    def execute(self):
        assert self.qbit_n > 0, "Qubit size should be at least 1"
        self.initialize_circuit()
        self.connect_qubits()
        self.execute_circuit()
        self.generate_result()


#### [step 1] - Create a quantum circuit


You have to choose the size of the qubit the circuit will use

In [0]:
# Choose the size of qubit size
qubit_size = 3
circuit = QuantumSimulator(qubit_size)

#### [step 2] - Add gates to qubit


Its time to apply quantum unitary gate to the qubits! There are total 9 gates we can use. Check out the following gates we can use :) You can add the gate to circuit by following command


    [1qubit gate] - specify gate type and qubit index
    circuit.[gate type]([qubit index])
    circuit.x(0)

    [2qubit gate] - specify gate type and control qubit / target qubit index
    circuit.[gate type]([control qubit index, target qubit index])
    circuit.cx(0, 1)


**1qubit gate**

$X_{gate} = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$
$Y_{gate} = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}$
$Z_{gate} = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$

$H_{gate} = \begin{pmatrix} \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}} \end{pmatrix}$
$T_{gate} = \begin{pmatrix} 1 & 0 \\ 0 & e^\frac{i\pi}{4} \end{pmatrix}$

**2qubit gate**

$CX_{gate} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix}$
$CY_{gate} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & -i \\ 0 & 0 & i & 0 \end{pmatrix}$

$CZ_{gate} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1 \end{pmatrix}$
$CH_{gate} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ 0 & 0 & \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}} \end{pmatrix}$

In [0]:
# Add the gate to the circuit
circuit.h(0) # Add gate H to qubit 0
circuit.h(1) # Add gate H to qubit 1
circuit.cx(0, 2) # Add gate CX from qubit 0 to qubit 2
circuit.cx(1, 2) # Add gate CX from qubit 1 to qubit 2

### 3. **How to check the result**

Now you can run the quantum simulator!

    circuit.execute()

And now the result will be stored inside the simulator

    The basic result will be stored in statevector with amplitude
    And also provided with 

    circuit.result - statevector with amplitude
    circuit.result_prob - statevector with probability

In [7]:
# Run the circuit
circuit.execute()

# Checkout the result
print(circuit.result)
print(circuit.result_prob)

{'000': (0.4999999999999999+0j), '011': (0.4999999999999999+0j), '101': (0.4999999999999999+0j), '110': (0.4999999999999999+0j)}
{'000': 0.2499999999999999, '011': 0.2499999999999999, '101': 0.2499999999999999, '110': 0.2499999999999999}


### 4. **Other useful features**

1. Check the gates on circuit
2. Clear the circuit
3. Change the qubit size

##### 1. **Check the gates on circuit**

    circuit.circuit

Checkout the circuit gate that is stored inside. 

    <TIP>
    You can directly put the gates to the circuit than it will be changed

In [8]:
circuit.circuit

[('h', 0), ('h', 1), ('cx', 0, 2), ('cx', 1, 2)]

##### 2. **Clear the circuit**

    circuit.clear_circuit()

This will remove all the gates inside your circuit

In [0]:
circuit.clear_circuit()

In [10]:
circuit.circuit

[]

In [11]:
# Run the circuit
circuit.execute()

# Checkout the result
print(circuit.result)
print(circuit.result_prob)

{'000': (1+0j)}
{'000': 1.0}


##### 3. **Change the qubit size**

    circuit.change_qubit_size(3)

This will remove all the gates inside your circuit

In [12]:
# Run the circuit
circuit.execute()

# Checkout the result
print(circuit.result)
print(circuit.result_prob)

{'000': (1+0j)}
{'000': 1.0}


In [0]:
circuit.change_qubit_size(20)

In [14]:
# Run the circuit
circuit.execute()

# Checkout the result
print(circuit.result)
print(circuit.result_prob)

{'00000000000000000000': (1+0j)}
{'00000000000000000000': 1.0}


## Quantum Simulator Test

1. Q1 - bell state
2. Q2 - swap
3. Q3 - gate cancelation
4. Basic test
    - T, CH test
    - 5 qubit test
5. Advanced test
    - Random circuit test with random qubit size and random gate size
6. Test results

In [0]:
class QuantumSimulator():
    """Quantum simulator with tensornetwork"""
    def __init__(self, qbit_n):
        """
        Args:
            qbit_n: The number of total qubit size
            circuit: Where to store gate
        """
        assert qbit_n > 0, "Qubit size should be at least 1"
        self.qbit_n = qbit_n
        self.circuit = []

    def initialize_circuit(self):
        """
        Initialize the circuit to execute

        Args:
            qbits: Store the first initialized qubits
            measures: Store the edge that used for measure for each qubit
        """
        # Initialize qubit
        self.qbits = [tn.Node(np.array([1 + 0j, 0 + 0j])) for _ in range(self.qbit_n)]

        # Storing the Edge node that will be used to measure for each qubit
        self.measures = [self.qbits[i][0] for i in range(self.qbit_n)]
        self.amplitude = None
        self.result = {}
        self.result_prob = {}
   
    def connect_qubits(self):
        """
        Make the seperate tensornetworks to one tensornetwork using
        2 qubit identity gate
        CI - Control Identity
             [[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]]
        """
        for i in range(self.qbit_n - 1):
            self.add_ci(i, i + 1)

    def x(self, qbit_i):
        """
        Add X gate to specific 1 qubit index 
        x(qbit_i=0)
        """
        self.circuit.append(('x', qbit_i))

    def y(self, qbit_i):
        """
        Add Y gate to specific 1 qubit index 
        y(qbit_i=5)
        """
        self.circuit.append(('y', qbit_i))

    def z(self, qbit_i):
        """
        Add Z gate to specific 1 qubit index 
        z(qbit_i=3)
        """
        self.circuit.append(('z', qbit_i))

    def h(self, qbit_i):
        """
        Add H gate to specific 1 qubit index 
        h(qbit_i=2)
        """
        self.circuit.append(('h', qbit_i))

    def t(self, qbit_i):
        """
        Add T gate to specific 1 qubit index 
        t(qbit_i=0)
        """
        self.circuit.append(('t', qbit_i))

    def cx(self, qbit_c, qbit_t):
        """
        Add CX gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        cx(qbit_c=0, qbit_t=1)
        """
        self.circuit.append(('cx', qbit_c, qbit_t))

    def cy(self, qbit_c, qbit_t):
        """
        Add CY gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        cy(qbit_c=3, qbit_t=2)
        """
        self.circuit.append(('cy', qbit_c, qbit_t))

    def cz(self, qbit_c, qbit_t):
        """
        Add CZ gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        cz(qbit_c=2, qbit_t=4)
        """
        self.circuit.append(('cz', qbit_c, qbit_t))

    def ch(self, qbit_c, qbit_t):
        """
        Add CH gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        ch(qbit_c=1, qbit_t=3)
        """
        self.circuit.append(('ch', qbit_c, qbit_t))

    def add_x(self, qbit_i):
        X = tn.Node(np.array([[0, 1],
                              [1, 0]]))
        self.measures[qbit_i] ^ X[0]
        self.measures[qbit_i] = X[1]

    def add_y(self, qbit_i):
        Y = tn.Node(np.array([[0, 1j],
                              [-1j, 0]]))
        self.measures[qbit_i] ^ Y[0]
        self.measures[qbit_i] = Y[1]

    def add_z(self, qbit_i):
        Z = tn.Node(np.array([[1, 0],
                              [0, -1]]))
        self.measures[qbit_i] ^ Z[0]
        self.measures[qbit_i] = Z[1]

    def add_h(self, qbit_i):
        h_f = 1/math.sqrt(2)
        H = tn.Node(np.array([[h_f, h_f],
                              [h_f, -h_f]]))
        self.measures[qbit_i] ^ H[0]
        self.measures[qbit_i] = H[1]

    def add_t(self, qbit_i):
        e_j_pi = math.e ** ((1j * math.pi) / 4)
        T = tn.Node(np.array([[1, 0],
                              [0, e_j_pi]]))
        self.measures[qbit_i] ^ T[0]
        self.measures[qbit_i] = T[1]

    def add_cx(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        cx = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 0, 1],
                       [0, 0, 1, 0]])
        CX = tn.Node(cx.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CX[0]
        self.measures[qbit_t] ^ CX[1]
        self.measures[qbit_c] = CX[2]
        self.measures[qbit_t] = CX[3]

    def add_cy(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        cy = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 0, 1j],
                       [0, 0, -1j, 0]])
        CY = tn.Node(cy.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CY[0]
        self.measures[qbit_t] ^ CY[1]
        self.measures[qbit_c] = CY[2]
        self.measures[qbit_t] = CY[3]


    def add_cz(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        cz = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, -1]])
        CZ = tn.Node(cz.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CZ[0]
        self.measures[qbit_t] ^ CZ[1]
        self.measures[qbit_c] = CZ[2]
        self.measures[qbit_t] = CZ[3]

    def add_ch(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
            h_f: hadamard factor
        """
        h_f = 1/math.sqrt(2)
        ch = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, h_f, h_f],
                       [0, 0, h_f, -h_f]])
        CH = tn.Node(ch.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CH[0]
        self.measures[qbit_t] ^ CH[1]
        self.measures[qbit_c] = CH[2]
        self.measures[qbit_t] = CH[3]

    def add_ci(self, qbit_c, qbit_t):
        """
        Control Identity matrix for connect
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        ci = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]])
        CI = tn.Node(ci.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CI[0]
        self.measures[qbit_t] ^ CI[1]
        self.measures[qbit_c] = CI[2]
        self.measures[qbit_t] = CI[3]

    def generate_result(self):
        for i in range(2 ** self.qbit_n):
            # Generate statevector
            statevector = ""
            for shift in range(self.qbit_n):
                statevector = str((i >> shift) & 1) + statevector
            
            # Check the amplitude
            amp = self.amplitude[self.bitstring_to_int(statevector)]
            amp = complex(str(amp))

            if amp == 0:
                continue

            # TODO: Figure out how to get value out of jax array
            self.result[statevector] = amp 
            self.result_prob[statevector] = (amp*amp.conjugate()).real

    def bitstring_to_int(self, bitstring):
        int_list = []
        for i in bitstring[::-1]:
            int_list.append(int(i))
        return tuple(int_list)

    def execute_circuit(self):
        """
        gate is structed as following
        1 qubit gate - (gate_name, qubit_index)
        2 qubit gate - (gate_name, qubit_control_index, qubit_target_index)
        """

        # Connect the qubits
        for gate in self.circuit:
            if gate[0] in ['cx', 'cy', 'cz', 'ch']:
                exec('self.add_{}({}, {})'.format(gate[0], gate[1], gate[2]))
            else:
                exec('self.add_{}({})'.format(gate[0], gate[1]))

        nodes = tn.reachable(self.qbits[0])
        self.amplitude = tn.contractors.greedy(nodes, output_edge_order=self.measures).tensor

    def clear_circuit(self):
        self.circuit = []

    def change_qubit_size(self, size):
        assert size > 0, "Qubit size should be at least 1"
        self.qbit_n = size

    def execute(self):
        assert self.qbit_n > 0, "Qubit size should be at least 1"
        self.initialize_circuit()
        self.connect_qubits()
        self.execute_circuit()
        self.generate_result()


### Q1 - bell state

In [16]:
circuit = QuantumSimulator(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.execute()
print(circuit.result)
print(circuit.result_prob)

{'00': (0.7071067811865475+0j), '11': (0.7071067811865475+0j)}
{'00': 0.4999999999999999, '11': 0.4999999999999999}


In [17]:
# Generate qubit
qubits = LineQubit.range(2)

# Count simulate
simulator = Simulator()
circuit = Circuit()
circuit.append(cirq.H(qubits[0]))
circuit.append(cirq.CX(qubits[0], qubits[1]))
circuit.append(cirq.measure(*qubits, key='m'))
result = simulator.run(circuit, repetitions=1000)
print(circuit)
print(result.histogram(key=['m']))

# Density simulate
circuit_d = Circuit()
circuit_d.append(cirq.H(qubits[0]))
circuit_d.append(cirq.CX(qubits[0], qubits[1]))
result_d = simulator.simulate(circuit_d)

print(np.around(result_d.final_state, 2))

0: ───H───@───M('m')───
          │   │
1: ───────X───M────────
Counter({0: 536, 3: 464})
[ 0.71+0.j  0.  +0.j -0.  +0.j  0.71+0.j]


### Q2 - swap

In [18]:
circuit = QuantumSimulator(2)
circuit.x(1)
circuit.cx(0, 1)
circuit.cx(1, 0)
circuit.cx(0, 1)
circuit.execute()
print(circuit.result)
print(circuit.result_prob)

{'01': (1+0j)}
{'01': 1.0}


In [19]:
# Generate qubit
qubits = LineQubit.range(2)

# Count simulate
simulator = Simulator()
circuit = Circuit()
circuit.append(cirq.X(qubits[1]))
circuit.append(cirq.CX(qubits[0], qubits[1]))
circuit.append(cirq.CX(qubits[1], qubits[0]))
circuit.append(cirq.CX(qubits[0], qubits[1]))
circuit.append(cirq.measure(qubits[0], key='m_1'))
circuit.append(cirq.measure(qubits[1], key='m_2'))
result = simulator.run(circuit, repetitions=1000)
print(circuit)
print(result.multi_measurement_histogram(keys=['m_1', 'm_2']))

# Density simulate
circuit_d = Circuit()
circuit_d.append(cirq.X(qubits[0]))
circuit_d.append(cirq.CX(qubits[0], qubits[1]))
circuit_d.append(cirq.CX(qubits[1], qubits[0]))
circuit_d.append(cirq.CX(qubits[0], qubits[1]))
result_d = simulator.simulate(circuit_d)

print(np.around(result_d.final_state, 2))

0: ───────@───X───@───M('m_1')───
          │   │   │
1: ───X───X───@───X───M('m_2')───
Counter({(1, 0): 1000})
[0.+0.j 1.+0.j 0.+0.j 0.+0.j]


### Q3 - gate cancelation

In [20]:
circuit = QuantumSimulator(3)

# connect
circuit.y(0)
circuit.h(1)
circuit.x(2)
circuit.cx(0, 1)
circuit.h(0)
circuit.cz(1, 2)
circuit.cy(0, 2)

# deconnect
circuit.cy(0, 2)
circuit.cz(1, 2)
circuit.h(0)
circuit.cx(0, 1)
circuit.x(2)
circuit.h(1)
circuit.y(0)
circuit.execute()
print(circuit.result)
print(circuit.result_prob)

{'000': (0.9999999999999996+0j), '001': (1.232595164407831e-32+0j), '010': (1.232595164407831e-32+0j), '011': (-1.232595164407831e-32+0j)}
{'000': 0.9999999999999991, '001': 1.5192908393215678e-64, '010': 1.5192908393215678e-64, '011': 1.5192908393215678e-64}


In [21]:
# Generate qubit
qubits = LineQubit.range(3)

# Count simulate
simulator = Simulator()
circuit = Circuit()
circuit.append(cirq.Y(qubits[0]))
circuit.append(cirq.H(qubits[1]))
circuit.append(cirq.X(qubits[2]))
circuit.append(cirq.CX(qubits[0], qubits[1]))
circuit.append(cirq.H(qubits[0]))
circuit.append(cirq.CZ(qubits[1], qubits[2]))
circuit.append(cirq.ControlledGate(cirq.Y)(qubits[0], qubits[2]))

circuit.append(cirq.ControlledGate(cirq.Y)(qubits[0], qubits[2]))
circuit.append(cirq.CZ(qubits[1], qubits[2]))
circuit.append(cirq.H(qubits[0]))
circuit.append(cirq.CX(qubits[0], qubits[1]))
circuit.append(cirq.X(qubits[2]))
circuit.append(cirq.H(qubits[1]))
circuit.append(cirq.Y(qubits[0]))
circuit.append(cirq.measure(qubits[0], key='m_1'))
circuit.append(cirq.measure(qubits[1], key='m_2'))
result = simulator.run(circuit, repetitions=1000)
print(circuit)
print(result.multi_measurement_histogram(keys=['m_1', 'm_2']))


# Density simulate
circuit_d = Circuit()
circuit_d.append(cirq.Y(qubits[0]))
circuit_d.append(cirq.H(qubits[1]))
circuit_d.append(cirq.X(qubits[2]))
circuit_d.append(cirq.CX(qubits[0], qubits[1]))
circuit_d.append(cirq.H(qubits[0]))
circuit_d.append(cirq.CZ(qubits[1], qubits[2]))
circuit_d.append(cirq.ControlledGate(cirq.Y)(qubits[0], qubits[2]))

circuit_d.append(cirq.ControlledGate(cirq.Y)(qubits[0], qubits[2]))
circuit_d.append(cirq.CZ(qubits[1], qubits[2]))
circuit_d.append(cirq.H(qubits[0]))
circuit_d.append(cirq.CX(qubits[0], qubits[1]))
circuit_d.append(cirq.X(qubits[2]))
circuit_d.append(cirq.H(qubits[1]))
circuit_d.append(cirq.Y(qubits[0]))
result_d = simulator.simulate(circuit_d)

print(np.around(result_d.final_state, 2))

0: ───Y───@───H───@───@───H───@───Y───M('m_1')───
          │       │   │       │
1: ───H───X───@───┼───┼───@───X───H───M('m_2')───
              │   │   │   │
2: ───X───────@───Y───Y───@───X──────────────────
Counter({(0, 0): 1000})
[ 1.-0.j  0.-0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j -0.+0.j]


### Basic test

#### T, CH gate test

In [22]:
circuit = QuantumSimulator(2)
circuit.t(0)
circuit.ch(0, 1)
circuit.execute()
print(circuit.result)
print(circuit.result_prob)

{'00': (1+0j)}
{'00': 1.0}


In [23]:
# Generate qubit
qubits = LineQubit.range(2)

# Count simulate
simulator = Simulator()
circuit = Circuit()
circuit.append(cirq.T(qubits[0]))
circuit.append(cirq.ControlledGate(cirq.H)(qubits[0], qubits[1]))
circuit.append(cirq.measure(qubits[0], key='m_1'))
circuit.append(cirq.measure(qubits[1], key='m_2'))
result = simulator.run(circuit, repetitions=1000)
print(circuit)
print(result.multi_measurement_histogram(keys=['m_1', 'm_2']))

# Density simulate
circuit_d = Circuit()
circuit_d.append(cirq.T(qubits[0]))
circuit_d.append(cirq.ControlledGate(cirq.H)(qubits[0], qubits[1]))
result_d = simulator.simulate(circuit_d)

print(np.around(result_d.final_state, 2))

0: ───T───@───M('m_1')───
          │
1: ───────H───M('m_2')───
Counter({(0, 0): 1000})
[ 1.+0.j  0.+0.j  0.+0.j -0.+0.j]


In [24]:
circuit = QuantumSimulator(2)
circuit.x(0)
circuit.ch(0, 1)
circuit.execute()
print(circuit.result)
print(circuit.result_prob)

{'01': (0.7071067811865475+0j), '11': (0.7071067811865475+0j)}
{'01': 0.4999999999999999, '11': 0.4999999999999999}


In [25]:
# Generate qubit
qubits = LineQubit.range(2)

# Count simulate
simulator = Simulator()
circuit = Circuit()
circuit.append(cirq.X(qubits[0]))
circuit.append(cirq.ControlledGate(cirq.H)(qubits[0], qubits[1]))
circuit.append(cirq.measure(qubits[0], key='m_1'))
circuit.append(cirq.measure(qubits[1], key='m_2'))
result = simulator.run(circuit, repetitions=1000)
print(circuit)
print(result.multi_measurement_histogram(keys=['m_1', 'm_2']))

# Density simulate
circuit_d = Circuit()
circuit_d.append(cirq.X(qubits[0]))
circuit_d.append(cirq.ControlledGate(cirq.H)(qubits[0], qubits[1]))
result_d = simulator.simulate(circuit_d)

print(np.around(result_d.final_state, 2))

0: ───X───@───M('m_1')───
          │
1: ───────H───M('m_2')───
Counter({(1, 0): 522, (1, 1): 478})
[0.  +0.j 0.  +0.j 0.71+0.j 0.71+0.j]


In [26]:
circuit = QuantumSimulator(2)
circuit.t(0)
circuit.y(0)
circuit.t(0)
circuit.ch(0, 1)
circuit.ch(1, 0)
circuit.execute()
print(circuit.result)
print(circuit.result_prob)

{'01': (-0.4999999999999999+0.5j), '10': (-0.3535533905932737+0.35355339059327373j), '11': (0.3535533905932737-0.35355339059327373j)}
{'01': 0.4999999999999999, '10': 0.24999999999999992, '11': 0.24999999999999992}


In [27]:
# Generate qubit
qubits = LineQubit.range(2)

# Count simulate
simulator = Simulator()
circuit = Circuit()
circuit.append(cirq.T(qubits[0]))
circuit.append(cirq.Y(qubits[0]))
circuit.append(cirq.T(qubits[0]))
circuit.append(cirq.ControlledGate(cirq.H)(qubits[0], qubits[1]))
circuit.append(cirq.ControlledGate(cirq.H)(qubits[1], qubits[0]))
circuit.append(cirq.measure(qubits[0], key='m_1'))
circuit.append(cirq.measure(qubits[1], key='m_2'))
result = simulator.run(circuit, repetitions=1000)
print(circuit)
print(result.multi_measurement_histogram(keys=['m_1', 'm_2']))

# Density simulate
density_simulator = cirq.DensityMatrixSimulator()
circuit_d = Circuit()
circuit_d.append(cirq.T(qubits[0]))
circuit_d.append(cirq.Y(qubits[0]))
circuit_d.append(cirq.T(qubits[0]))
circuit_d.append(cirq.ControlledGate(cirq.H)(qubits[0], qubits[1]))
circuit_d.append(cirq.ControlledGate(cirq.H)(qubits[1], qubits[0]))
result_d = simulator.simulate(circuit_d)

print(np.around(result_d.final_state, 2))

0: ───T───Y───T───@───H───M('m_1')───
                  │   │
1: ───────────────H───@───M('m_2')───
Counter({(1, 0): 510, (1, 1): 262, (0, 1): 228})
[ 0.  -0.j   -0.35+0.35j -0.5 +0.5j   0.35-0.35j]


#### 5 qubit test

In [28]:
circuit = QuantumSimulator(5)
circuit.h(0)
circuit.h(1)
circuit.h(2)
circuit.h(3)
circuit.cx(0, 1)
circuit.t(0)
circuit.y(0)
circuit.ch(4, 0)
circuit.ch(1, 3)
circuit.cy(2, 3)
circuit.execute()
print(circuit.result)
print(circuit.result_prob)

{'00000': (0.1767766952966368-0.17677669529663684j), '00001': 0.24999999999999992j, '00010': (0.2499999999999999-0.24999999999999992j), '00011': 0.35355339059327356j, '00100': (-0.17677669529663684-0.1767766952966368j), '00101': (0.24999999999999992+0j), '00110': (5.592785792689345e-18+5.592785792689343e-18j), '00111': (-7.909393519468831e-18+0j), '01000': (0.1767766952966368-0.17677669529663684j), '01001': 0.24999999999999992j, '01010': (-5.592785792689343e-18+5.592785792689345e-18j), '01011': -7.909393519468831e-18j, '01100': (0.17677669529663684+0.1767766952966368j), '01101': (-0.24999999999999992+0j), '01110': (0.24999999999999992+0.2499999999999999j), '01111': (-0.35355339059327356+0j)}
{'00000': 0.06249999999999996, '00001': 0.06249999999999996, '00010': 0.1249999999999999, '00011': 0.12499999999999986, '00100': 0.06249999999999996, '00101': 0.06249999999999996, '00110': 6.255850584581555e-35, '00111': 6.255850584581555e-35, '01000': 0.06249999999999996, '01001': 0.06249999999999

In [29]:
# Generate qubit
qubits = LineQubit.range(5)

# Count simulate
simulator = Simulator()
circuit = Circuit()
circuit.append(cirq.H(qubits[0]))
circuit.append(cirq.H(qubits[1]))
circuit.append(cirq.H(qubits[2]))
circuit.append(cirq.H(qubits[3]))
circuit.append(cirq.CX(qubits[0], qubits[1]))
circuit.append(cirq.T(qubits[0]))
circuit.append(cirq.Y(qubits[0]))
circuit.append(cirq.ControlledGate(cirq.H)(qubits[4], qubits[0]))
circuit.append(cirq.ControlledGate(cirq.H)(qubits[1], qubits[3]))
circuit.append(cirq.ControlledGate(cirq.Y)(qubits[2], qubits[3]))
circuit.append(cirq.measure(qubits[0], key='m_1'))
circuit.append(cirq.measure(qubits[1], key='m_2'))
circuit.append(cirq.measure(qubits[2], key='m_3'))
circuit.append(cirq.measure(qubits[3], key='m_4'))
circuit.append(cirq.measure(qubits[4], key='m_5'))
result = simulator.run(circuit, repetitions=1000)
print(circuit)
print(result.multi_measurement_histogram(keys=['m_1', 'm_2', 'm_3', 'm_4', 'm_5']))

# Density simulate
density_simulator = cirq.DensityMatrixSimulator()
circuit_d = Circuit()
circuit_d.append(cirq.H(qubits[0]))
circuit_d.append(cirq.H(qubits[1]))
circuit_d.append(cirq.H(qubits[2]))
circuit_d.append(cirq.H(qubits[3]))
circuit_d.append(cirq.CX(qubits[0], qubits[1]))
circuit_d.append(cirq.T(qubits[0]))
circuit_d.append(cirq.Y(qubits[0]))
circuit_d.append(cirq.ControlledGate(cirq.H)(qubits[4], qubits[0]))
circuit_d.append(cirq.ControlledGate(cirq.H)(qubits[1], qubits[3]))
circuit_d.append(cirq.ControlledGate(cirq.Y)(qubits[2], qubits[3]))
result_d = simulator.simulate(circuit_d)

print(np.around(result_d.final_state, 2))

                             ┌─────────┐
0: ───H───@───T───Y───────────H────────────M('m_1')───
          │                   │
1: ───H───X───@───M('m_2')────┼───────────────────────
              │               │
2: ───H───────┼───@───────────┼M('m_3')───────────────
              │   │           │
3: ───H───────H───Y───────────┼M('m_4')───────────────
                              │
4: ───────────────────────────@────────────M('m_5')───
                             └─────────┘
Counter({(0, 1, 0, 0, 0): 143, (1, 1, 0, 0, 0): 129, (0, 1, 1, 1, 0): 124, (1, 1, 1, 1, 0): 114, (1, 0, 1, 0, 0): 72, (0, 0, 1, 0, 0): 71, (1, 0, 1, 1, 0): 69, (0, 0, 1, 1, 0): 64, (0, 0, 0, 1, 0): 59, (1, 0, 0, 0, 0): 57, (0, 0, 0, 0, 0): 54, (1, 0, 0, 1, 0): 44})
[ 0.18-0.18j  0.  +0.j    0.18-0.18j  0.  +0.j   -0.18-0.18j  0.  +0.j
  0.18+0.18j  0.  +0.j    0.25-0.25j  0.  +0.j    0.  +0.j    0.  +0.j
  0.  -0.j   -0.  +0.j    0.25+0.25j  0.  +0.j    0.  +0.25j -0.  +0.j
  0.  +0.25j  0.  +0.j    0.25-0.j  

In [0]:
circuit.append(cirq.H(qubits[0]))

### Advanced Test

#### Random circuit test with random qubit size and random gate size


In [0]:
class QuantumSimulatorTest():
    """Test the quantum simulator"""
    def __init__(self,
                 min_qubit_size=1,
                 max_qubit_size=5,
                 min_gate_size=1,
                 max_gate_size=40):
        """
        Choose the testing enviroment
        Args:
            min_qubit_size: minimum number of the qubit size
            min_qubit_size: maximum number of the qubit size
            min_qubit_size: minimum number of the gate size
            min_qubit_size: maximum number of the gate size
        """
        self.max_qubit_size = max_qubit_size
        self.min_qubit_size = min_qubit_size
        self.max_gate_size = max_gate_size
        self.min_gate_size = min_gate_size

    def __call__(self,
                 quantum_simulator,
                 backend,
                 test_size=10,
                 error_path=None,
                 error_file=None):
        """
        Process test
        Quantum simulator testing using google cirq
        Args:
            quantum_simulator: quantum simulator that will be used to simulate
                - qasm_simulator: show the number of shots for each statevector
                - statevector_simualtor: show the amplitude and prob for each statevector
                                         measure gates are disabled
            test_size: total test case size
            error_path: where to store the error case
            error_file: name of the error file
        """
        # Get cirq backend type
        if backend == 'count' or backend == 'density':
            self.cirq_backend = backend
        else:
            raise NotImplementedError('Only Count or Density backend is supported')
        
        self.cirq_simulator = cirq.Simulator()

        # Check the correct
        total = 0
        correct = 0
        test_start_time = time.time()

        # Store the error case
        if error_path:
            if not(os.path.isdir(error_path)):
                os.makedirs(error_path)
            f = open(os.path.join(error_path, error_file), 'w')

        # Run the test!
        for test_i in tqdm(range(test_size)):
            # Generate the random circuit and qasm(Google cirq)
            random_qubit_size = random.randint(self.min_qubit_size, self.max_qubit_size)
            random_gate_size = random.randint(self.min_gate_size, self.max_gate_size)
            circuit, qasm = self.random_circuit(random_qubit_size, random_gate_size)

            # Print out the current test case information
            print(c.BOLD)
            print('TEST CASE [{} / {}] qubit: {} gate: {}'.format(test_i + 1,
                                                                  test_size,
                                                                  random_qubit_size,
                                                                  random_gate_size))
            print(c.ENDC)

            # Run the Quantum simulator and get the result
            qs_start_time = time.time()
            quantum_simulator.change_qubit_size(random_qubit_size)
            quantum_simulator.circuit = circuit
            quantum_simulator.execute()
            qs_end_time = time.time()

            self.qs_result = quantum_simulator.result_prob

            # Run Google cirq simulator and get the result
            shots = 1000 # default

            cirq_start_time = time.time()
            cirq_circuit = circuit_from_qasm(qasm)
            if self.cirq_backend == 'count':
                self.cirq_result = simulator.run(cirq_circuit, repetitions=shots)
            elif self.cirq_backend == 'density':
                self.cirq_result = simulator.simulate(cirq_circuit)
            cirq_end_time = time.time()

            ############################################################
            ### PRINT OUT THE INFORMATIONS #############################
            ############################################################
            
            # 1. Circuit information
            print(circuit)
            print(cirq_circuit)

            # 2. Quantum simulation
            print('')
            print(c.BOLD + '[ QUANTUM SIMULATOR ]' + c.ENDC)
            print(c.CURL, end="")
            print("Quantum simulator tested time: %s sec\n" % format(qs_end_time - qs_start_time))
            print(c.ENDC, end="")
            print(quantum_simulator.result)
            print(self.qs_result)
            print('')

            # 3. Cirq simulation
            print(c.BOLD + '[ GOOGLE CIRQ ]' + c.ENDC)
            print(c.CURL, end="")
            
            print("Cirq tested time: %s sec\n" % format(cirq_end_time - cirq_start_time))
            print(c.ENDC, end="")
            if self.cirq_backend == 'count':
                self.preprocess_cirq_count(random_qubit_size)
                print(self.cirq_count)
            if self.cirq_backend == 'density':
                self.generate_cirq_result(random_qubit_size)
                print(self.cirq_amp)
                print(self.cirq_prob)
            print('')

            # 4. Compare the result
            print(c.BOLD + '[ COMPARE ]' + c.ENDC)
            cmp_start_time = time.time()
            error_n, cmp_total, total_error = self.compare_probability(shots)
            print(c.CURL, end="")
            print("Comparing time: %s sec" % format(time.time() - cmp_start_time))
            print(c.ENDC)
            if error_n == 0:
                print(c.OKBLUE + 'ERROR OCCUR / TOTAL CHECK: ' + c.ENDC, error_n, '/', cmp_total)
                print(c.OKBLUE + 'TOTAL ERROR: ' + c.ENDC, total_error,'\n')
            else:
                print(c.FAIL + 'ERROR OCCUR / TOTAL CHECK: ' + c.ENDC, error_n, '/', cmp_total)
                print(c.FAIL + 'TOTAL ERROR: ' + c.ENDC, total_error,'\n')

            ############################################################
            ############################################################
            ############################################################

            # Update the error information and save the error case to file
            if error_n == 0:
                correct += 1
            elif error_path and self.cirq_backend == 'count':
                error_case = ""
                error_case += str(circuit) + '\n'
                error_case += '< Quantum simulator probability >\n'
                error_case += str(self.qs_result) +'\n'
                error_case += '< Cirq statevector count information >\n'
                error_case += str(self.cirq_count) + '\n'
                error_case += 'Error occur / Total check: '
                error_case += str(error_n) + '/' + str(cmp_total) + '\n'
                error_case += 'Total error: '
                error_case += str(total_error) + '\n\n'
                f.write(error_case)
            elif error_path and self.cirq_backend == 'density':
                error_case = ""
                error_case += str(circuit) + '\n'
                error_case += '< Quantum simulator amplitude & probability >\n'
                error_case += str(quantum_simulator.result) +'\n'
                error_case += str(self.qs_result) +'\n'
                error_case += '< Cirq amplitude & probability >\n'
                error_case += str(self.cirq_amp) + '\n'
                error_case += str(self.cirq_prob) + '\n'
                error_case += 'Error occur / Total check: '
                error_case += str(error_n) + '/' + str(cmp_total) + '\n'
                error_case += 'Total error: '
                error_case += str(total_error) + '\n\n'
                f.write(error_case)
            total += 1

        # Close the error case file
        if error_path:
            f.close()
        
        # Print out the final result
        if correct == total:
            print(c.OKBLUE + "The final result is {} / {}".format(correct, total) + c.ENDC)
        else:
            print(c.FAIL + "The final result is {} / {}".format(correct, total) + c.ENDC)
        print("Total tested time: %s sec" % format(time.time() - test_start_time))
    
    def preprocess_cirq_count(self, qubit_size):
        self.cirq_count = {}
        keys = []
        for i in range(qubit_size):
            keys.append('m_{}'.format(i))
        cirq_count = self.cirq_result.multi_measurement_histogram(keys=keys)
        for statevector, count in cirq_count.items():
             # Generate statevector
            bitstring = ""
            for bit in statevector:
                bitstring = str(bit) + bitstring
            self.cirq_count[bitstring] = count

    def generate_cirq_result(self, qubit_size):
        """
        Generate statevector dictionary stores probability of cirq simulator
        based on amplitude from cirq statevector simulator
        """
        cirq_amplitude = self.cirq_result.final_state
        self.cirq_amp = {}
        self.cirq_prob = {}
        for i in range(2 ** qubit_size):
            # Generate statevector
            statevector = ""
            for shift in range(qubit_size):
                statevector = str((i >> shift) & 1) + statevector
            statevector = statevector[::-1]
            
            # Check the amplitude
            amp = cirq_amplitude[i]

            if amp == 0:
                continue

            self.cirq_amp[statevector] = amp 
            self.cirq_prob[statevector] = (amp*amp.conjugate()).real
    
    def compare_probability(self, shots, delta=1e-6):
        """
        Check each statevector probability and if the value is
        more than give delta(default: 1e-6) value consider as error
        """
        total_error = 0
        error_n = 0
        cmp_total = 0

        # Get statevector info from both simulator
        if self.cirq_backend == 'count':
            cirq_count = self.cirq_count
            state_vectors = list(set(self.qs_result.keys()).union(set(cirq_count.keys())))
        if self.cirq_backend == 'density':
            state_vectors = list(set(self.qs_result.keys()).union(set(self.cirq_amp.keys())))
        
        # Check is there is an error
        for state_vector in state_vectors:
            qs_val = self.qs_result.get(state_vector, 0)

            if self.cirq_backend == 'count':
                cirq_val = cirq_count.get(state_vector, 0)/shots
            if self.cirq_backend == 'density':
                cirq_val = self.cirq_prob.get(state_vector, 0)

            error = abs(qs_val - cirq_val)
            total_error += error
            if error > delta:
                error_n += 1
            cmp_total += 1

        return error_n, cmp_total, total_error

    def random_circuit(self, qubit_size=5, gate_size=20):
        """
        Args:
            gate_1: list of 1 qubit gates
            gate_2: list of 2 qubit gates
            qubit_list: list of qubit index
        """
        # Quantum Simulator
        gate_1 = ['x', 'y', 'z', 'h', 't']
        gate_2 = ['cx', 'cy', 'cz', 'ch']
        qubit_list = list(range(qubit_size))

        circuit = []
        for _ in range(gate_size):
            if qubit_size == 1 or random.random() > 0.5:
                circuit.append(tuple([random.choice(gate_1), *random.sample(qubit_list, 1)]))
            else:
                circuit.append(tuple([random.choice(gate_2), *random.sample(qubit_list, 2)]))

        # Google cirq
        qasm = self.circuit_to_qasm(circuit, qubit_size)
        
        return circuit, qasm

    def circuit_to_qasm(self, circuit, qubit_size):
        # Add meta data
        qasm_text = 'OPENQASM 2.0;\ninclude "qelib1.inc";\n'
        
        # Add quantum bits and classical bits
        qasm_text += 'qreg q[{0}];\n'.format(qubit_size)
        if self.cirq_backend == 'count':
            qasm_text += 'creg m[{0}];\n'.format(qubit_size)

        # Add Identity gate to all qubit to prevent qubit dissapearing
        # When converting qasm to ciruit
        for i in range(qubit_size):
            qasm_text += 'id q[{0}];\n'.format(i)

        # Add quantum gates
        for gate in circuit:
            # 1 qubit gate
            if len(gate) == 2:
                qasm_text += '{0} q[{1}];\n'.format(gate[0], gate[1])
            # 2 qubit gate
            elif len(gate) == 3:
                qasm_text += '{0} q[{1}],q[{2}];\n'.format(gate[0], gate[1], gate[2])
            else:
                raise NotImplementedError("Quantum simulator doesn't support qubit gates more than 2")
        
        # Add measure gate
        if self.cirq_backend == 'count':
            for qbit_i in range(qubit_size):
                qasm_text += 'measure q[{0}] -> m[{0}];\n'.format(qbit_i)
        
        return qasm_text


#### **Quantum Simulator Test**

1. Choose the random simulator qubit & gate range


    min_qubit_size - Size of minumun qubit size to test
    max_qubit_size - Size of maximum qubit size to test
    min_gate_size - Size of minumun gate size to test
    max_gate_size - Size of maximum gate size to test

2. Test the simulator


    quantum_simulator - Built quantum simulator to be tested
    test_size - Choose how many cases you want to test
    error_path - Choose the path where error case file will be saved
    error_file - Choose the error case file name 

In [0]:
t = QuantumSimulatorTest(min_qubit_size=1,
                         max_qubit_size=20,
                         min_gate_size=1,
                         max_gate_size=100)
circuit = QuantumSimulator(1)

In [35]:
t(quantum_simulator=circuit,
  backend='density',
  test_size=50,
  error_path='/error',
  error_file='error_case.txt')


TEST CASE [1 / 50] qubit: 17 gate: 30

[('y', 8), ('x', 9), ('h', 5), ('x', 0), ('y', 6), ('ch', 11, 3), ('ch', 16, 0), ('cy', 2, 14), ('y', 12), ('t', 4), ('ch', 16, 2), ('h', 12), ('y', 15), ('cy', 1, 11), ('ch', 10, 8), ('cy', 16, 9), ('cy', 12, 10), ('y', 1), ('y', 6), ('cx', 15, 3), ('ch', 6, 10), ('y', 0), ('ch', 2, 0), ('y', 4), ('h', 16), ('ch', 13, 12), ('cy', 2, 10), ('cy', 0, 5), ('x', 13), ('cz', 11, 12)]
             ┌───┐   ┌────┐   ┌──┐   ┌──┐   ┌──┐
q_0: ────I────X───────H────────Y──────H───────@────
                      │               │       │
q_1: ────I────────────┼@───────Y──────┼───────┼────
                      ││              │       │
q_2: ────I──────@─────┼┼───────H──────@──────@┼────
                │     ││       │             ││
q_3: ────I─────H┼─────┼┼─X─────┼─────────────┼┼────
               ││     ││ │     │             ││
q_4: ────I────T┼┼─────┼┼Y┼─────┼─────────────┼┼────
               ││     ││ │     │             ││
q_5: ────I────H┼┼─────┼┼─┼───

### Test Results

#### Test case

#### Test case that proves minus (-) of Y is opposite

$Y_{gate} = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}$
$Y_{fixed} = \begin{pmatrix} 0 & i \\ -i & 0 \end{pmatrix}$

$CY_{gate} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & -i \\ 0 & 0 & i & 0 \end{pmatrix}$
$CY_{fixed} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & i \\ 0 & 0 & -i & 0 \end{pmatrix}$

In [0]:
case = [('y', 1), ('ch', 1, 0), ('cy', 0, 1), ('t', 0), ('cx', 1, 0), ('h', 1)]

In [0]:
qasm = t.circuit_to_qasm(case, 2)
qasm

In [0]:
circuit = circuit_from_qasm(qasm)
print(circuit)

In [0]:
circuit = QuantumSimulator(2)
circuit.circuit = case
circuit.execute()
print(circuit.result)
print(circuit.result_prob)

In [0]:
# Count simulate
simulator = Simulator()
result = simulator.run(circuit, repetitions=1000)
print(result.multi_measurement_histogram(keys=['m_0', 'm_1']))